# Endless Runner Q-table training

Heavily inspired on the Q-table Colab notebook from Huggingface's DeepRL course: https://huggingface.co/learn/deep-rl-course/unit2/hands-on

## Import the packages

In [2]:
import gymnasium as gym
import GameEnvs

## Create the environment

In [3]:
env = gym.make("EndlessRunner-v0")

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


## Understanding our environment

In [4]:
print("Action space:", env.action_space)
print("Sample action:", env.action_space.sample())
print("Observation space:", env.observation_space)
print("Sample observation:", env.observation_space.sample())

Action space: Discrete(3)
Sample action: 2
Observation space: Dict('obstacle_x': Box(50.0, 800.0, (1,), float32), 'obstacle_y': Box(266.66666, 300.0, (1,), float32), 'player_y': Box(300.0, 321.0, (1,), float32))
Sample observation: OrderedDict({'obstacle_x': array([506.74197], dtype=float32), 'obstacle_y': array([268.97168], dtype=float32), 'player_y': array([314.36783], dtype=float32)})


: 

## Initialize the Q-table
Need to discretize the observations to make the Q-table. The solution seems to be to bin the observations, the question is how.

We have the obstacle's x position, the obstacles y positon and the player's y positon.

Obstacle x: Goes from being at x = 800 to x = 50 (behind the player)
Obstacle y: is either at the low (300) or high position (266.66666)
Player y: goes from floor level (300) to max jump height (195)